In [1]:
import os
from time import time
import yaml

from argparse import ArgumentParser, Namespace

from s3ts.api.nets.wrapper import WrapperModel
from s3ts.helper_functions import load_dm, str_time

from pytorch_lightning import Trainer
from pytorch_lightning.utilities.model_summary import summarize

In [2]:
from s3ts.helper_functions import get_model

In [3]:
import torch
import numpy as np

In [4]:
hparam_path = os.path.join("./training_cv/img_HARTH_3_5_32_2_bs128_lr0.001_cnn_gap20_mlp32_1_p32_r0.1_syn300/version_0/", "hparams.yaml")
train_args = None

with open(hparam_path, "r") as f:
    try:
        train_args = Namespace(**eval(yaml.safe_load(f)["args"]))
    except yaml.YAMLError as exc:
        print(exc)

In [5]:
train_args.weight_decayL1 = 0
train_args.weight_decayL2 = 0
train_args.cached = True

In [6]:
dm = load_dm(train_args)

Loaded dataset HARTH with a total of 6215293 observations for window size 32
Using synthetic shapes...
Computing dissimilarity frames...
Loading cached dissimilarity frames if available...
Using 4272507 observations for training, 1592033 for validation and 350753 observations for test


In [7]:
model = get_model("test", train_args, dm)

Input shape:  torch.Size([1, 18, 32, 32])
Latent shape:  torch.Size([1, 80])
[tensor(80), 32, 12]


In [9]:
for n, p in model.named_parameters():
    print(n, p.shape)

encoder.cnn_0.conv.weight torch.Size([20, 18, 5, 5])
encoder.cnn_0.conv.bias torch.Size([20])
encoder.cnn_0.bn.weight torch.Size([20])
encoder.cnn_0.bn.bias torch.Size([20])
encoder.cnn_1.conv.weight torch.Size([20, 20, 4, 4])
encoder.cnn_1.conv.bias torch.Size([20])
encoder.cnn_1.bn.weight torch.Size([20])
encoder.cnn_1.bn.bias torch.Size([20])
encoder.cnn_2.conv.weight torch.Size([40, 20, 3, 3])
encoder.cnn_2.conv.bias torch.Size([40])
encoder.cnn_2.bn.weight torch.Size([40])
encoder.cnn_2.bn.bias torch.Size([40])
encoder.last.conv.weight torch.Size([80, 40, 1, 1])
encoder.last.conv.bias torch.Size([80])
decoder.fcn_layer_0.weight torch.Size([32, 80])
decoder.fcn_layer_0.bias torch.Size([32])
decoder.fcn_out.weight torch.Size([12, 32])
decoder.fcn_out.bias torch.Size([12])


In [10]:
sum(p.square().sum() for name, p in model.named_parameters() if ("bias" not in name and "bn" not in name))

tensor(68.5660, grad_fn=<AddBackward0>)

In [11]:
sum(p.abs().sum() for name, p in model.named_parameters() if ("bias" not in name and "bn" not in name))

tensor(1093.2843, grad_fn=<AddBackward0>)